## Demonstrates some common TensorFlow errors

This notebook demonstrates some common TensorFlow errors, how to find them, and how to fix them.

In [8]:
import tensorflow as tf
print(tf.__version__)

1.8.0


# Shape error

In [ ]:
def some_method(data):
  a = data[:,0:2]
  c = data[:,1]
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
  print sess.run(some_method(fake_data))

In [ ]:
def some_method(data):
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
  print sess.run(some_method(fake_data))

In [9]:
def some_method(data):
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1:3]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
  print sess.run(some_method(fake_data))

(4, 2)
(4, 2)
[[12.884487 11.878131 12.449096 15.721323]
 [11.878131 10.962208 11.489996 14.509306]
 [12.449096 11.489996 12.043255 15.207892]
 [15.721323 14.509306 15.207892 19.204166]]


In [10]:
import tensorflow as tf

x = tf.constant([[3, 2],
                 [4, 5],
                 [6, 7]])
print "x.shape", x.shape

# http://docs.w3cub.com/tensorflow~python/tf/expand_dims/
expanded = tf.expand_dims(x, 1)
print "expanded.shape", expanded.shape

# https://www.quora.com/How-does-tf-slice-work-in-TensorFlow
sliced = tf.slice(x, [0, 1], [2, 1])
print "sliced.shape", sliced.shape

with tf.Session() as sess:
  print "expanded: ", expanded.eval()
  print "sliced: ", sliced.eval()

x.shape (3, 2)
expanded.shape (3, 1, 2)
sliced.shape (2, 1)
expanded:  [[[3 2]]

 [[4 5]]

 [[6 7]]]
sliced:  [[2]
 [5]]


# Vector vs scalar

In [ ]:
def some_method(data):
  print data.get_shape()
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1:3]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([5.0, 3.0, 7.1])
  print sess.run(some_method(fake_data))

In [11]:
def some_method(data):
  print data.get_shape()
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1:3]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([5.0, 3.0, 7.1])
  fake_data = tf.expand_dims(fake_data, 0)
  print sess.run(some_method(fake_data))

(1, 3)
(1, 2)
(1, 2)
[[12.884487]]


# Type error

In [ ]:
def some_method(a, b):
  s = (a + b)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 4, 5],
      [2, 8, 7]
    ])
  print sess.run(some_method(fake_a, fake_b))

In [13]:
def some_method(a, b):
  b = tf.cast(b, tf.float32)
  s = (a + b)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 4, 5],
      [2, 8, 7]
    ])
  print sess.run(some_method(fake_a, fake_b))

[[15.633617 16.0493  ]
 [16.0493   17.439611]]


# TensorFlow debugger

Wrap your normal Session object with tf_debug.LocalCLIDebugWrapperSession

In [14]:
import tensorflow as tf
from tensorflow.python import debug as tf_debug

def some_method(a, b):
  b = tf.cast(b, tf.float32)
  s = (a / b)
  s2 = tf.matmul(s, tf.transpose(s))
  return tf.sqrt(s2)

with tf.Session() as sess:
  fake_a = [
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ]
  fake_b = [
      [2, 0, 5],
      [2, 8, 7]
    ]
  a = tf.placeholder(tf.float32, shape=[2, 3])
  b = tf.placeholder(tf.int32, shape=[2, 3])
  k = some_method(a, b)
  
  # Note: won't work without the ui_type="readline" argument because
  # Datalab is not an interactive terminal and doesn't support the default "curses" ui_type.
  # If you are running this a standalone program, omit the ui_type parameter and add --debug
  # when invoking the TensorFlow program
  #      --debug (e.g: python debugger.py --debug )
  sess = tf_debug.LocalCLIDebugWrapperSession(sess, ui_type="readline")
  sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
  print sess.run(k, feed_dict = {a: fake_a, b: fake_b})

run-start: run #1: 1 fetch (Sqrt_5:0); 2 feeds

TTTTTT FFFF DDD  BBBB   GGG 
  TT   F    D  D B   B G    
  TT   FFF  D  D BBBB  G  GG
  TT   F    D  D B   B G   G
  TT   F    DDD  BBBB   GGG 

Session.run() call #1:

Fetch(es):
  Sqrt_5:0

Feed dict:
  Placeholder:0
  Placeholder_1:0

Select one of the following commands to proceed ---->
  run:
    Execute the run() call with debug tensor-watching
  run -n:
    Execute the run() call without debug tensor-watching
  run -t <T>:
    Execute run() calls (T - 1) times without debugging, then execute run() once more with debugging and drop back to the CLI
  run -f <filter_name>:
    Keep executing run() calls until a dumped tensor passes a given, registered filter (conditional breakpoint mode)
    Registered filter(s):
        * has_inf_or_nan
  invoke_stepper:
    Use the node-stepper interface, which allows you to interactively step through nodes involved in the graph run() call and inspect/modify their values

For more details, see help

In the tfdbg> window that comes up, try the following:
* run -f has_inf_or_nan
* Notice that several tensors are dumped once the filter criterion is met
* List the inputs to a specific tensor:
* li transpose:0 
* Print the value of a tensor
* pt transpose:0
* Where is the inf?

Visit https://www.tensorflow.org/programmers_guide/debugger for usage details of tfdbg

## tf.Print()

In [15]:
%writefile debugger.py
import tensorflow as tf

# tf.Print needs to be used in standalone .py file
def some_method(a, b):
  b = tf.cast(b, tf.float32)
  s = (a / b)
  print_ab = tf.Print(s, [a, b])
  s = tf.where(tf.is_nan(s), print_ab, s)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 0, 5],
      [2, 8, 7]
    ])
  
  print sess.run(some_method(fake_a, fake_b))

Writing debugger.py


In [16]:
!python debugger.py

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2018-07-11 04:07:12.742517: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
[[5 3 7.1]...][[2 0 5]...]
[[      inf       inf]
 [      inf 1.4336528]]
